<a href="https://colab.research.google.com/github/lgregs/CrewTeste/blob/main/crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREW AI
* Gerenciamento de Agentes de IA

## Preparação do Ambiente
* Instalações, Libs e Variáveis Globais.

In [1]:
!pip install uv

In [2]:
!poetry install --no-root

/bin/bash: line 1: poetry: command not found


In [3]:
!pip install -q crewai crewai_tools

In [4]:
#!crewai create crew SmartReader

In [5]:
#!crewai

In [6]:
pip freeze > requirements.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from crewai import Agent, Task, Crew, LLM
import os
from google.colab import userdata
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, PDFSearchTool, FileReadTool, JSONSearchTool

In [25]:
# Criar os diretorios caso não existam ...
#os.makedirs("/content/paths/pdf", exist_ok=True)
#os.makedirs("/content/paths/txt", exist_ok=True)
#os.makedirs("/content/paths/json", exist_ok=True)



Arquivos na pasta PDF: ['PO 144899.pdf', 'Pedido 11398_24 Lar - Friendly Mate.pdf']


In [8]:
# Variaveis Globais
caminho_pdf = "/content/drive/MyDrive/CrewPaths/pdf"
caminho_txt ="/content/drive/MyDrive/CrewPaths/txt"
caminho_json = "/content/drive/MyDrive/CrewPaths/json"
dados = " 'Nome do Vendedor', 'Endereço do Vendedor', 'Nome do Comprador', 'Endereço do Comprador', 'Número da Ordem', 'Data da Ordem', 'Descrição do Produto', 'Quantidade', 'Valor Unitário', 'Valor Total' "

In [ ]:
# Verificar se os pdfs estao na pasta
print("Arquivos na pasta PDF:", os.listdir("/content/drive/MyDrive/CrewPaths/pdf"))

### Api Keys

In [9]:
from dotenv import load_dotenv
load_dotenv()

False

In [10]:
# OPEN AI (LLM)
os.environ["AZURE_API_KEY"] = userdata.get('AZURE_API_KEY')
os.environ["AZURE_API_BASE"] = userdata.get('AZURE_API_BASE')
os.environ["AZURE_API_VERSION"] = userdata.get('AZURE_API_VERSION')

#os.environ["SERPER_API_KEY"] = userdata.get('SERPERKEY')
# SERPER (Motor de pesquisas)

In [11]:
os.environ["OPENAI_API_KEY"] = "chave"
os.environ["OPENAI_API_BASE"] = "url"
os.environ["OPENAI_API_VERSION"] = "2024-08-01-preview"
os.environ["OPENAI_DEPLOYMENT_NAME"] = "azure/gpt-4o"  # Nome do deployment

In [12]:
from crewai import LLM

llm = LLM(
    model="azure/gpt-4",
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42,
    base_url="url",
    api_key="chave",
    api_version="2024-08-01-preview"

)

### TOOLS

In [13]:
# Tools utilizadas.
#search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
pdf_tool = PDFSearchTool()
file_tool = FileReadTool()
json_tool = JSONSearchTool()

## Funções

## Agentes

In [14]:
PDFReaderAgent = Agent(
    role='Especialista em leitura de arquivos PDF.',
    goal="Ler o PDF e extrair texto do PDF da ordem de compra em {caminhoPdf}.",
    backstory="Com vasta experiência em processamento de documentos, o PDFReaderAgent é especialista em extrair informações precisas e relevantes de arquivos PDF.",
    tools=[pdf_tool, file_tool],
    llm=llm,
    verbose=True

)

In [15]:
TextFileWriterAgent = Agent(
    role='Especialista em Salvar Arquivos de Texto',
    goal="Salvar o texto extraído do PDF em um arquivo de texto em {caminhoTxt}",
    backstory="TextFileWriterAgent é conhecido por sua precisão e eficiência em manipulação de arquivos de texto,"
    "garantindo que os dados sejam salvos corretamente para uso futuro.",
    tools=[file_tool, json_tool],
    llm=llm,
    verbose=True

)

In [16]:
DataExtractorAgent = Agent(
    role='Especialista em Extração de Dados',
    goal="Extrair informações específicas do texto da ordem de compra salvo no arquivo de texto em {caminhoTxt}",
    backstory="DataExtractorAgent tem uma habilidade especial para identificar e extrair dados importantes de grandes volumes de texto,"
    "tornando-o indispensável para tarefas de análise de dados",
    tools=[file_tool, json_tool],
    llm=llm,
    verbose=True

)

In [17]:
JSONWriterAgent = Agent(
    role='Especialista em Estruturação de Dados',
    goal="Estruturar as informações extraídas em um arquivo JSON em {caminhoTxt}",
    backstory="Com uma paixão por organização e estruturação de dados, JSONWriterAgent garante que todas as informações sejam apresentadas de forma clara e acessível.",
    tools=[file_tool, json_tool],
    llm=llm,
    verbose=True

)

### Tasks

In [18]:
read_pdf_task = Task(
    description = (
        "Ler e extrair todo o texto do aquivo PDF da ordem de compra utilizando o caminho fornecido {caminhoPdf}"
        "Garantir que todo o texto foi extraído para futuro processamento."
    ),
    agent=PDFReaderAgent,
    expected_output="Realizar a extração de todo o texto dos arquivos PDF.",
    verbose=True
)

In [19]:
saveTxt = Task(
    description=(
        "Salvar todo o texto extraído pelo agente readPdf em um arquivo txt localizado em {caminhTtxt}."
    ),
    agent=TextFileWriterAgent,
    expected_output="Um arquivo de Texto contendo todo o texto estraído do PDF salvo no diretório {caminhoTxt}. ",
    verbose=True
)

In [20]:
extract_data_task = Task(
    description = (
        "Identificar e extrair as seguintes informações específicas do texto salvo em {caminhoTxt}: {tema}."
),
    agent=DataExtractorAgent,
    expected_output="Informações especificas extraídas e organizadas em um formato estruturado",
    verbose=True)

In [21]:
write_json_task = Task(
    description = (
        "Estruturar as informações extraídas em um arquivo JSON no caminho {caminhoJson}"
    ),
    agent = JSONWriterAgent,
    expected_output="Arquivo JSON contendo as informações extraídas e estruturadas.",
    verbose=True
)

## CREW SETUP

In [22]:
equipe = Crew(
    agents=[PDFReaderAgent, TextFileWriterAgent, DataExtractorAgent, JSONWriterAgent],
    tasks=[read_pdf_task, saveTxt, extract_data_task, write_json_task],
    verbose=True
)

In [23]:
# data = variavel contendo os valores que devem ser retirados do texto dos pdfs
entradas = {"tema": dados,"caminhoPdf":caminho_pdf,"caminhoTxt":caminho_txt, "caminhoJson": caminho_json}
#resultado = equipe.kickoff(inputs=entradas)

In [26]:
def validar_arquivo(caminho):
    if not os.path.exists(caminho):
        print(f"Erro: Arquivo não encontrado - {caminho}")
        return False
    return True

In [29]:
# ATUALIZAÇÃO DO FORMATO DE EXECUÇÃO DO PROJETO

def processar_pdfs():
    # Listar arquivos PDF
    arquivos_pdf = [f for f in os.listdir(caminho_pdf) if f.endswith('.pdf')]

    for arquivo in arquivos_pdf:
        current_file = os.path.join(caminho_pdf, arquivo)
        print(f"Processando: {current_file}")

        # Validar existência do arquivo
        if not validar_arquivo(current_file):
            continue

        # Configurar entradas para o Crew
        entradas = {
            "tema": dados,
            "caminhoPdf": current_file,
            "caminhoTxt": os.path.join(caminho_txt, f"{os.path.splitext(arquivo)[0]}.txt"),
            "caminhoJson": os.path.join(caminho_json, f"{os.path.splitext(arquivo)[0]}.json"),
        }

        # Executar Crew
        try:
            resultado = equipe.kickoff(inputs=entradas)
            print(f"Resultado para {arquivo}: {resultado}")
        except Exception as e:
            print(f"Erro ao processar {arquivo}: {str(e)}")

processar_pdfs()


Processando: /content/paths/pdf/PO 144899.pdf
# Agent: Especialista em leitura de arquivos PDF.
## Task: Ler e extrair todo o texto do aquivo PDF da ordem de compra utilizando o caminho fornecido /content/paths/pdf/PO 144899.pdfGarantir que todo o texto foi extraído para futuro processamento.


# Agent: Especialista em leitura de arquivos PDF.
## Thought: Thought: Eu preciso ler o conteúdo do arquivo PDF especificado para extrair todo o texto. Vou utilizar a ferramenta "Read a file's content" para realizar essa ação.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"/content/paths/pdf/PO 144899.pdf\"}"
## Tool Output: 
Fail to read the file /content/paths/pdf/PO 144899.pdf. Error: 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}, 'pdf': {'description': 'Mandatory pdf path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a PDF's content.



# Agent: Especialista em leitura de arquivos PDF.
## Thought: Thought: Como a tentativa de ler o arquivo diretamente falhou, vou tentar usar a ferramenta "Search a PDF's content" para buscar o conteúdo do PDF.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \" \", \"pdf\": \"/content/paths/pdf/PO 144899.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() mis

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}, 'pdf': {'description': 'Mandatory pdf path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a PDF's content.



# Agent: Especialista em leitura de arquivos PDF.
## Thought: Thought: Vou tentar novamente usar a ferramenta "Search a PDF's content" para buscar qualquer conteúdo do PDF.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \".\", \"pdf\": \"/content/paths/pdf/PO 144899.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() missing 2 required keyword-only argument

...

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}, 'pdf': {'description': 'Mandatory pdf path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a PDF's content.



# Agent: Especialista em leitura de arquivos PDF.
## Thought: Thought: Ocorreu um erro ao tentar ler o arquivo diretamente, então vou tentar buscar o conteúdo do PDF utilizando a ferramenta de busca semântica para garantir a extração do texto.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \" \", \"pdf\": \"/content/paths/pdf/Pedido 11398_24 Lar - Friendly Mate.pdf\"}"
## Tool Output: 

I encountered an error while trying to use

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


 

I encountered an error while trying to use the tool. This was the error: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}, 'pdf': {'description': 'Mandatory pdf path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a PDF's content.



# Agent: Especialista em leitura de arquivos PDF.
## Thought: Thought: Vou tentar usar a ferramenta de busca semântica novamente, desta vez com uma consulta diferente para ver se consigo extrair o conteúdo do PDF.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"ordem de compra\", \"pdf\": \"/content/paths/pdf/Pedido 11398_24 Lar - Friendly Mate.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This 

KeyboardInterrupt: 

In [35]:
## teste isolado

In [37]:
from crewai_tools import PDFSearchTool

pdf_tool = PDFSearchTool()
query = "Nome do Vendedor"
pdf_path = "/content/paths/pdf/Pedido 11398_24 Lar - Friendly Mate.pdf"

try:
    resultado = pdf_tool.run({"query": query, "pdf": pdf_path})
    print("Resultado da busca no PDF:", resultado)
except Exception as e:
    print(f"Erro ao usar a ferramenta: {e}")


Using Tool: Search a PDF's content
Erro ao usar a ferramenta: Expected document to be a str, got {'query': 'Nome do Vendedor', 'pdf': '/content/paths/pdf/Pedido 11398_24 Lar - Friendly Mate.pdf'} in query.
